In [1]:
import pandas as pd
df=pd.read_csv('medals.csv',error_bad_lines=False)

C:\Users\purus\AppData\Local\Temp\ipykernel_8408\4050561555.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('medals.csv',error_bad_lines=False)


In [2]:
df.drop('country_code', axis=1, inplace=True)
df.drop('country_3_letter_code', axis=1, inplace=True)
df.drop('participant_title', axis=1, inplace=True)
df.drop('athlete_url', axis=1, inplace=True)
column_names = df.columns.tolist()
print(column_names)

['discipline_title', 'slug_game', 'event_title', 'event_gender', 'medal_type', 'participant_type', 'athlete_full_name', 'country_name']


In [3]:

def create_discipline_dataframes(df):
    # Get unique disciplines
    unique_disciplines = df['discipline_title'].unique()

    # Create a dictionary to store DataFrames for each discipline
    discipline_dataframes = {}

    # Iterate over unique disciplines
    for discipline in unique_disciplines:
        # Filter the DataFrame for the current discipline
        discipline_df = df[df['discipline_title'] == discipline]

        # Store the discipline DataFrame in the dictionary
        discipline_dataframes[discipline] = discipline_df

    return discipline_dataframes

discipline_dataframes=create_discipline_dataframes(df)

###GENERAL CASE EXTRACTION ON THE BASIS OF DISCIPLINE, SLUG,EVENT_TITLE

In [5]:
#Issue with medal count
def get_discipline_dataframe(discipline_dataframes, discipline):
    if discipline in discipline_dataframes:
        return discipline_dataframes[discipline]
    else:
        print(f"No DataFrame found for discipline: {discipline}")
        return None

def extract_slug_game_dataframe(dataframe, slug):
    slug_game_df = dataframe[dataframe["slug_game"] == slug]
    if not slug_game_df.empty:
        return slug_game_df
    else:
        print(f"No DataFrame found for slug game: {slug}")
        return None

def get_max_medal_countries(df):
    """
    Filter the DataFrame to get the countries with the maximum medals for each event title.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.

    Returns:
        pandas.DataFrame: The filtered DataFrame with the countries that achieved the maximum medals
                          for each event title and medal type combination.
    """
    # Group the DataFrame by 'event_title', 'country_name', and 'medal_type' columns
    grouped_df = df.groupby(['event_title', 'country_name', 'medal_type']).size().reset_index(name='medal_count')

    # Filter the grouped DataFrame to get the countries with the maximum medals for each event title and medal type
    max_medal_countries = grouped_df[grouped_df['medal_count'] == grouped_df['medal_count'].max()]

    # Extract the event titles from the DataFrame
    event_titles = max_medal_countries['event_title'].unique()

    # Filter the DataFrame for the extracted event titles
    filtered_df = max_medal_countries[max_medal_countries['event_title'].isin(event_titles)]

    return filtered_df


def calculate_most_medals(df, medal_type, event_title):
    """
    Calculate the countries with the most medals of a specific medal type for a given event title.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        medal_type (str): The medal type to calculate the most medals for.
        event_title (str): The event title to consider.

    Returns:
        pandas.DataFrame: The DataFrame with the countries that achieved the most medals of the specified type for the given event title.
    """
    event_df = df[df['event_title'] == event_title]
    medal_df = event_df[event_df['medal_type'] == medal_type]
    most_medals = medal_df.groupby('country_name').size().reset_index(name=f'{medal_type}_medals').sort_values(by=f'{medal_type}_medals', ascending=False)
    return most_medals

discipline = input("Enter The Discipline to be concidered:")
DF = get_discipline_dataframe(discipline_dataframes, discipline)

slug=input("Enter the Venue of the Olympics:")
DF_2=extract_slug_game_dataframe(DF,slug)

result_df = get_max_medal_countries(DF_2)
result_df.head(20)

MEDAL_TYPE = input("Enter the medal type: ")
EVENT_TITLE = input("Enter the event title: ")

mdf = result_df[result_df['medal_type'] == MEDAL_TYPE]
most_medals = calculate_most_medals(mdf, MEDAL_TYPE, EVENT_TITLE)

print("Country that won", MEDAL_TYPE, "medal for", EVENT_TITLE, "event:")
most_medals.head()

Country that won GOLD medal for Boxing event:


,country_name,GOLD_medals


### MEN AND WOMEN SPORT PARTICIPATION ANALYSIS

In [6]:
import pandas as pd
import numpy as np

def analyze_gender_orientation(df):
    """
    Analyze the gender orientation of disciplines based on the proportion of male and female competitors.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.

    Returns:
        pandas.DataFrame: The DataFrame with the gender orientation analysis for each discipline.
    """
    # Remove rows with null values in the 'event_gender' column
    df = df.dropna(subset=['event_gender'])

    # Group the DataFrame by 'discipline_title' and 'event_gender'
    grouped_df = df.groupby(['discipline_title', 'event_gender']).size().reset_index(name='competitor_count')

    # Pivot the DataFrame to get the count of male and female competitors for each discipline
    pivot_df = grouped_df.pivot(index='discipline_title', columns='event_gender', values='competitor_count')

    # Calculate the proportion of male and female competitors for each discipline
    pivot_df['total_competitors'] = pivot_df['Men'].fillna(0) + pivot_df['Women'].fillna(0)
    pivot_df['male_proportion'] = pivot_df['Men'].fillna(0) / pivot_df['total_competitors']
    pivot_df['female_proportion'] = pivot_df['Women'].fillna(0) / pivot_df['total_competitors']

    # Categorize disciplines based on gender orientation
    conditions = [
        (pivot_df['male_proportion'] > 0.5),
        (pivot_df['female_proportion'] > 0.5),
        (pivot_df['male_proportion'] == pivot_df['female_proportion']),
        (pivot_df['Men'].isna() & pivot_df['Women'].notna()),
        (pivot_df['Women'].isna() & pivot_df['Men'].notna())
    ]
    choices = ['Male-Oriented', 'Female-Oriented', 'Balanced', 'Female-Oriented', 'Male-Oriented']
    pivot_df['orientation'] = pd.Series(np.select(conditions, choices, default=None), index=pivot_df.index)

    # Filter in 'Balanced' disciplines
    filtered_df = pivot_df[(pivot_df['orientation'] == 'Balanced')]
    male_df = pivot_df[~pivot_df['orientation'].isin(['Balanced', 'Female-Oriented'])]
    female_df= pivot_df[~pivot_df['orientation'].isin(['Balanced', 'Male-Oriented'])]
    return filtered_df,male_df,female_df

# Assuming you have the DataFrame named 'df'
balanced_orientation_df,male_orientation_df,female_orientation_df = analyze_gender_orientation(df)

In [7]:
import plotly.graph_objects as go

def plot_gender_orientation(filtered_df, male_df, female_df):
    """
    Plot the gender orientation of disciplines using Plotly.

    Parameters:
        filtered_df (pandas.DataFrame): The DataFrame containing the filtered disciplines.
        male_df (pandas.DataFrame): The DataFrame containing the male-oriented disciplines.
        female_df (pandas.DataFrame): The DataFrame containing the female-oriented disciplines.
    """
    # Create a scatter plot with labels for the gender orientations
    fig = go.Figure()

    # Add a scatter trace for balanced orientation
    fig.add_trace(
        go.Scatter(
            x=filtered_df.index,
            y=filtered_df['total_competitors'],
            mode='markers',
            hovertemplate='<b>Discipline:</b> %{x}<br><b>Orientation:</b> Balanced',
            marker=dict(color='black', symbol='circle', size=10),
            name='Balanced'
        )
    )

    # Add a scatter trace for male orientation
    fig.add_trace(
        go.Scatter(
            x=male_df.index,
            y=male_df['total_competitors'],
            mode='markers',
            hovertemplate='<b>Discipline:</b> %{x}<br><b>Orientation:</b> Male-Oriented',
            marker=dict(color='blue', symbol='circle', size=10),
            name='Male-Oriented'
        )
    )

    # Add a scatter trace for female orientation
    fig.add_trace(
        go.Scatter(
            x=female_df.index,
            y=female_df['total_competitors'],
            mode='markers',
            hovertemplate='<b>Discipline:</b> %{x}<br><b>Orientation:</b> Female-Oriented',
            marker=dict(color='pink', symbol='circle', size=10),
            name='Female-Oriented'
        )
    )

    # Update the layout
    fig.update_layout(
        title='Gender Orientation of Disciplines',
        xaxis_title='Discipline',
        yaxis_title='Total Competitors',
        showlegend=True
    )

    # Show the interactive plot
    fig.show()

# Assuming you have the filtered, male-oriented, and female-oriented DataFrames
plot_gender_orientation(balanced_orientation_df, male_orientation_df, female_orientation_df)


###participate vs medals(Checking to see if higher participation in an event correlates to high medal count)

In [8]:
import pandas as pd

def determine_most_participated_sport(df, country):
    """
    Determine the most participated sport for a given country.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country (str): The country for which to determine the most participated sport.

    Returns:
        str: The most participated sport for the given country.
    """
    # Group the DataFrame by 'country_name' and 'discipline_title' and count the occurrences
    country_sport_count = df.groupby(['country_name', 'discipline_title']).size().reset_index(name='participation_count')

    # Pivot the DataFrame to have 'country_name' as the index and 'discipline_title' as the columns
    pivot_df = pd.pivot_table(country_sport_count, index='country_name', columns='discipline_title', values='participation_count', fill_value=0)

    # Determine the most participated sport for the given country
    most_participated_sport = pivot_df.loc[country].idxmax()

    return most_participated_sport

def determine_best_medals_by_discipline(df, country):
    """
    Determine the countries with the best overall medal count in each discipline for a given country.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country (str): The country for which to determine the best overall medal count.

    Returns:
        pandas.DataFrame: A DataFrame showing the countries with the best overall medal count in each discipline.
    """
    # Group the DataFrame by 'country_name', 'discipline_title', and 'medal_type' and count the occurrences
    medal_count = df.groupby(['country_name', 'discipline_title', 'medal_type']).size().reset_index(name='medal_count')

    # Determine the total medal count for each country in each discipline
    total_medals = medal_count.groupby(['country_name', 'discipline_title'])['medal_count'].sum().reset_index()

    # Determine the country with the best overall medal count in each discipline
    best_medals_country = total_medals.groupby('discipline_title')['medal_count'].idxmax()
    country_best_medals_df = total_medals.loc[best_medals_country, ['country_name', 'discipline_title']]
    country_best_medals_df.sort_values(by='country_name', inplace=True)

    return country_best_medals_df[country_best_medals_df['country_name']==country]

# Assuming df is your DataFrame
country = input("Enter the Country to be concidered:")  # Specify the country of interest
most_participated_sport = determine_most_participated_sport(df, country)

print(f"The most participated sport by {country} contestestants is: {most_participated_sport}")
country_best_medals = determine_best_medals_by_discipline(df, country)

print(f"The dsicipline/disciplines for which the country of {country} has the highest medal count:")
country_best_medals.head(50)


The most participated sport by United States of America contestestants is: Athletics
The dsicipline/disciplines for which the country of United States of America has the highest medal count:


,country_name,discipline_title
1659,United States of America,Roque
1620,United States of America,Athletics
1623,United States of America,Basketball
1626,United States of America,Boxing
1632,United States of America,Cycling BMX
1641,United States of America,Equestrian Jumping
1640,United States of America,Equestrian Eventing
1674,United States of America,Swimming
1660,United States of America,Rowing
1643,United States of America,Figure skating


In [9]:
fig = go.Figure()

# Add a bar for the most participated sport
fig.add_trace(go.Bar(
    x=[most_participated_sport],
    y=[1],
    name='Most Participated Sport',
    marker_color='blue'
))

# Add bars for the disciplines with the highest medal count
fig.add_trace(go.Bar(
    x=country_best_medals['discipline_title'],
    y=[1] * len(country_best_medals),
    name='Highest Medal Count',
    marker_color='green'
))

# Update the layout
fig.update_layout(
    title=f"Sports Analysis for {country}",
    xaxis_title='Discipline',
    yaxis_title='',
    showlegend=True,
    barmode='group'
)

# Show the interactive plot
fig.show()


In [10]:
import pandas as pd
import plotly.graph_objects as go

def determine_most_participated_sport(df, country):
    """
    Determine the most participated sport for a given country.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country (str): The country for which to determine the most participated sport.

    Returns:
        str: The most participated sport for the given country.
    """
    # Group the DataFrame by 'country_name' and 'discipline_title' and count the occurrences
    country_sport_count = df.groupby(['country_name', 'discipline_title']).size().reset_index(name='participation_count')

    # Pivot the DataFrame to have 'country_name' as the index and 'discipline_title' as the columns
    pivot_df = pd.pivot_table(country_sport_count, index='country_name', columns='discipline_title', values='participation_count', fill_value=0)

    # Determine the most participated sport for the given country
    most_participated_sport = pivot_df.loc[country].idxmax()

    return most_participated_sport

def determine_best_medals_by_discipline(df, country):
    """
    Determine the countries with the best overall medal count in each discipline for a given country.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country (str): The country for which to determine the best overall medal count.

    Returns:
        pandas.DataFrame: A DataFrame showing the countries with the best overall medal count in each discipline.
    """
    # Group the DataFrame by 'country_name', 'discipline_title', and 'medal_type' and count the occurrences
    medal_count = df.groupby(['country_name', 'discipline_title', 'medal_type']).size().reset_index(name='medal_count')

    # Determine the total medal count for each country in each discipline
    total_medals = medal_count.groupby(['country_name', 'discipline_title'])['medal_count'].sum().reset_index()

    # Determine the country with the best overall medal count in each discipline
    best_medals_country = total_medals.groupby('discipline_title')['medal_count'].idxmax()
    country_best_medals_df = total_medals.loc[best_medals_country, ['country_name', 'discipline_title', 'medal_count']]
    country_best_medals_df.sort_values(by='country_name', inplace=True)

    return country_best_medals_df[country_best_medals_df['country_name']==country]

# Assuming df is your DataFrame
country = input("Enter the Country to be considered:")  # Specify the country of interest
most_participated_sport = determine_most_participated_sport(df, country)
country_best_medals = determine_best_medals_by_discipline(df, country)

# Calculate the count of participants for the most participated sport
participants_count = df[df['discipline_title'] == most_participated_sport].groupby('country_name').size().get(country, 0)

fig = go.Figure()

# Add a bar for the most participated sport
fig.add_trace(go.Bar(
    x=[most_participated_sport],
    y=[participants_count],
    name='Most Participated Sport',
    marker_color='blue',
    text=[participants_count],
    textposition='auto',
    hovertemplate=f"Discipline: {most_participated_sport}<br>Participants: {participants_count}",
    showlegend=True
))

# Add bars for the disciplines with the highest medal count
for _, row in country_best_medals.iterrows():
    discipline = row['discipline_title']
    medals_count = row['medal_count']
    fig.add_trace(go.Bar(
        x=[discipline],
        y=[medals_count],
        name=f'Highest Medal Count - {discipline}',
        marker_color='green',
        text=[medals_count],
        textposition='auto',
        hovertemplate=f"Discipline: {discipline}<br>Medals: {medals_count}",
        showlegend=True
    ))

# Update the layout
fig.update_layout(
    title=f"Sports Analysis for {country}",
    xaxis_title='Discipline',
    yaxis_title='Count',
    showlegend=True,
    barmode='group'
)

# Show the interactive plot
fig.show()


### Highest number of participants from a country for a given discipline in a given slug_game(Venue)

In [11]:
def get_country_with_highest_count(df, discipline_name, slug_name):
    """
    Get the country with the highest count for a given discipline name and slug name.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        discipline_name (str): The discipline name to filter.
        slug_name (str): The slug name to filter.

    Returns:
        pandas.DataFrame: A DataFrame with the country with the highest count for the given discipline name and slug name.
    """
    # Grouping by 'slug_game', 'discipline_title', and 'country_name' and calculating the country counts
    grouped_df = df.groupby(['slug_game', 'discipline_title'])['country_name'].value_counts()

    # Extracting the country with the highest count for each slug game and discipline
    result = grouped_df.groupby(level=[0, 1]).idxmax().reset_index()
    result.columns = ['slug_game', 'discipline_title', 'country_name']

    # Filtering by discipline name and slug name
    result = result[(result['discipline_title'] == discipline_name) & (result['slug_game'] == slug_name)]

    return result

# Assuming df is your DataFrame
discipline_name = input("Enter the Discipline name:")
slug_name = input("Enter the venue of the Olympics:")
result = get_country_with_highest_count(df, discipline_name, slug_name)
result.head(10)

,slug_game,discipline_title,country_name
983,tokyo-2020,Boxing,"(tokyo-2020, Boxing, Great Britain)"


###countries preformance for a given discipline for across all venues

In [12]:
import pandas as pd
import plotly.graph_objects as go

def track_country_performance(df, country_name, discipline_title):
    """
    Track the performance of a country for a given discipline across all slug games.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country_name (str): The country name to track performance.
        discipline_title (str): The discipline title to track performance.

    Returns:
        pandas.DataFrame: A DataFrame with the total medals won for the given country and discipline in each slug game.
    """
    # Filter the DataFrame for the specified country and discipline
    filtered_df = df[(df['country_name'] == country_name) & (df['discipline_title'] == discipline_title)]

    # Group the filtered DataFrame by 'slug_game' and calculate the total medals won
    grouped_df = filtered_df.groupby('slug_game')['medal_type'].count().reset_index(name='total_medals')

    return grouped_df


def create_slider_plot(df, country_name, discipline_title):
    """
    Create a Plotly visualization with a slider for tracking the performance of a country in a discipline across slug games.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country_name (str): The country name to track performance.
        discipline_title (str): The discipline title to track performance.
    """
    # Track the performance of the country for the given discipline across all slug games
    performance_df = track_country_performance(df, country_name, discipline_title)

    # Create a figure
    fig = go.Figure()

    # Add an initial bar trace for the first slug game
    fig.add_trace(go.Bar(
        x=[performance_df['slug_game'].values[0]],
        y=[performance_df['total_medals'].values[0]],
        name='Total Medals',
    ))

    # Set the layout
    fig.update_layout(
        title=f'Performance of {country_name} in {discipline_title} as per total medal count',
        xaxis_title='Slug Game',
        yaxis_title='Total Medals',
        barmode='stack',
    )

    # Add a slider
    fig.update_layout(
        sliders=[
            {
                'currentvalue': {'prefix': 'Slug Game: '},
                'steps': [
                    {'method': 'animate', 'args': [[f'frame{i}']] ,'label': f'{game}'} for i, game in enumerate(performance_df['slug_game'])
                ],
                'transition': {'duration': 500, 'easing': 'linear'},
                'x': 0.1, 'len': 0.9,
                'y': 0, 'yanchor': 'top',
                'pad': {'t': 50, 'b': 10},
                'active': 0,
            }
        ],
    )

    # Add frames for each slug game
    frames = []
    for i, slug_game in enumerate(performance_df['slug_game']):
        frame = go.Frame(
            name=f'frame{i}',
            data=[go.Bar(x=[slug_game], y=[performance_df.loc[performance_df['slug_game'] == slug_game, 'total_medals'].values[0]])]
        )
        frames.append(frame)

    fig.frames = frames

    # Show the interactive plot
    fig.show()


# Assuming df is your DataFrame
country_name = input("Enter the country of Interest:")
discipline_title = input("Enter the discipline:")

create_slider_plot(df, country_name, discipline_title)


In [13]:
import pandas as pd
import plotly.graph_objects as go

def track_country_performance(df, country_name, discipline_title):
    """
    Track the performance of a country for a given discipline across all slug games.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country_name (str): The country name to track performance.
        discipline_title (str): The discipline title to track performance.

    Returns:
        pandas.DataFrame: A DataFrame with the total medals won for the given country and discipline in each slug game.
    """
    # Filter the DataFrame for the specified country and discipline
    filtered_df = df[(df['country_name'] == country_name) & (df['discipline_title'] == discipline_title)]

    # Group the filtered DataFrame by 'slug_game' and calculate the total medals won
    grouped_df = filtered_df.groupby('slug_game')['medal_type'].value_counts().unstack(fill_value=0)

    return grouped_df


def create_slider_plot(df, country_name, discipline_title):
    """
    Create a Plotly visualization with a slider for tracking the performance of a country in a discipline across slug games.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country_name (str): The country name to track performance.
        discipline_title (str): The discipline title to track performance.
    """
    # Track the performance of the country for the given discipline across all slug games
    performance_df = track_country_performance(df, country_name, discipline_title)

    # Create a figure
    fig = go.Figure()

    # Add bar traces for each medal type
    for medal_type in ['GOLD', 'SILVER', 'BRONZE']:
        fig.add_trace(go.Bar(
            x=performance_df.index,
            y=performance_df[medal_type],
            name=medal_type.capitalize(),
        ))

    # Set the layout
    fig.update_layout(
        title=f'Performance of {country_name} in {discipline_title} as per medal distribution',
        xaxis_title='Slug Game',
        yaxis_title='Total Medals',
        barmode='stack',
    )

    # Add a slider
    fig.update_layout(
        sliders=[
            {
                'currentvalue': {'prefix': 'Slug Game: '},
                'steps': [
                    {'method': 'animate', 'args': [[f'frame{i}']] ,'label': f'{game}'} for i, game in enumerate(performance_df.index)
                ],
                'transition': {'duration': 500, 'easing': 'linear'},
                'x': 0.1, 'len': 0.9,
                'y': 0, 'yanchor': 'top',
                'pad': {'t': 50, 'b': 10},
                'active': 0,
            }
        ],
    )

    # Add frames for each slug game
    frames = []
    for i, slug_game in enumerate(performance_df.index):
        frame_data = [
            go.Bar(x=[slug_game], y=[performance_df.loc[slug_game, 'GOLD']], name='Gold', marker=dict(color='gold')),
            go.Bar(x=[slug_game], y=[performance_df.loc[slug_game, 'SILVER']], name='Silver', marker=dict(color='silver')),
            go.Bar(x=[slug_game], y=[performance_df.loc[slug_game, 'BRONZE']], name='Bronze', marker=dict(color='peru'))
        ]
        frame = go.Frame(
            name=f'frame{i}',
            data=frame_data
        )
        frames.append(frame)

    fig.frames = frames

    # Show the interactive plot
    fig.show()


# Assuming df is your DataFrame
country_name = input("Enter the country of Interest:")
discipline_title = input("Enter the discipline:")

create_slider_plot(df, country_name, discipline_title)


In [14]:
import pandas as pd
import plotly.graph_objects as go

def track_country_performance(df, country_name, discipline_title):
    """
    Track the performance of a country for a given discipline across all slug games.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country_name (str): The country name to track performance.
        discipline_title (str): The discipline title to track performance.

    Returns:
        pandas.DataFrame: A DataFrame with the total medals won for the given country and discipline in each slug game.
    """
    # Filter the DataFrame for the specified country and discipline
    filtered_df = df[(df['country_name'] == country_name) & (df['discipline_title'] == discipline_title)]

    # Group the filtered DataFrame by 'slug_game' and calculate the total medals won
    grouped_df = filtered_df.groupby('slug_game')['medal_type'].count().reset_index(name='total_medals')

    return grouped_df


def create_slider_plot(df, country1_name, country2_name, discipline_title):
    """
    Create a Plotly visualization with a slider for tracking the performance of two countries in a discipline across slug games.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        country1_name (str): The name of the first country.
        country2_name (str): The name of the second country.
        discipline_title (str): The discipline title to track performance.
    """
    # Track the performance of the first country for the given discipline across all slug games
    performance_country1 = track_country_performance(df, country1_name, discipline_title)

    # Track the performance of the second country for the given discipline across all slug games
    performance_country2 = track_country_performance(df, country2_name, discipline_title)

    # Merge the performance DataFrames to include all slug games
    performance_combined = performance_country1.merge(performance_country2, on='slug_game', suffixes=('_country1', '_country2'))

    # Create a figure
    fig = go.Figure()

    # Add initial bar traces for the first slug game
    fig.add_trace(go.Bar(
        x=[performance_combined['slug_game'].values[0]],
        y=[performance_combined['total_medals_country1'].values[0]],
        name=country1_name,
        marker_color='blue',
        offsetgroup=0,
    ))
    fig.add_trace(go.Bar(
        x=[performance_combined['slug_game'].values[0]],
        y=[performance_combined['total_medals_country2'].values[0]],
        name=country2_name,
        marker_color='green',
        offsetgroup=1,
    ))

    # Set the layout
    fig.update_layout(
        title=f'Performance of {country1_name} vs {country2_name} in {discipline_title}',
        xaxis_title='Slug Game',
        yaxis_title='Total Medals',
        barmode='group',
    )

    # Add a slider
    fig.update_layout(
        sliders=[
            {
                'currentvalue': {'prefix': 'Slug Game: '},
                'steps': [
                    {'method': 'animate', 'args': [[f'frame{i}']], 'label': game} for i, game in enumerate(performance_combined['slug_game'])
                ],
                'transition': {'duration': 500, 'easing': 'linear'},
                'x': 0.1, 'len': 0.9,
                'y': 0, 'yanchor': 'top',
                'pad': {'t': 50, 'b': 10},
                'active': 0,
            }
        ],
    )

    # Add frames for each slug game
    frames = []
    for i, slug_game in enumerate(performance_combined['slug_game']):
        frame = go.Frame(
            name=f'frame{i}',
            data=[
                go.Bar(x=[slug_game], y=[performance_combined.loc[performance_combined['slug_game'] == slug_game, 'total_medals_country1'].values[0]], name=country1_name, marker_color='blue', offsetgroup=0),
                go.Bar(x=[slug_game], y=[performance_combined.loc[performance_combined['slug_game'] == slug_game, 'total_medals_country2'].values[0]], name=country2_name, marker_color='green', offsetgroup=1),
            ],
        )
        frames.append(frame)

    fig.frames = frames

    # Show the interactive plot
    fig.show()


# Assuming df is your DataFrame
country1_name = input("Enter the first country: ")
country2_name = input("Enter the second country: ")
discipline_title = input("Enter the discipline title: ")

create_slider_plot(df, country1_name, country2_name, discipline_title)


###Historic Performance of a particular olympian across all slugs

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go


def create_medal_count_plot(name, df):
    # Clone the DataFrame and filter for the specific athlete
    df_clone = df.copy()
    df_clone = df_clone.dropna(subset=['athlete_full_name'])
    df_clone = df_clone[df_clone['athlete_full_name'] == name]

    # Select the desired columns
    filtered_df = df_clone[['slug_game', 'event_title', 'athlete_full_name', 'medal_type']]

    # Group the DataFrame by 'slug_game' and 'medal_type' and count the occurrences
    grouped_df = filtered_df.groupby(['slug_game', 'medal_type']).size().unstack().fillna(0)

    # Create a list of medal types
    medal_types = grouped_df.columns

    # Create a trace for each medal type
    traces = []
    for medal_type in medal_types:
        trace = go.Bar(
            x=grouped_df.index,
            y=grouped_df[medal_type],
            name=medal_type
        )
        traces.append(trace)

    # Create the layout
    layout = go.Layout(
        title=f"Medal Count by Venue Name for {name}",
        xaxis=dict(title='Venue Name'),
        yaxis=dict(title='Medal Count'),
        barmode='stack'
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)

    # Display the interactive plot
    fig.show()

name =input()
create_medal_count_plot(name,df)

In [17]:
import pandas as pd
import plotly.graph_objects as go

def create_medal_count_plot(name, df):
    # Clone the DataFrame and filter for the specific athlete
    df_clone = df.copy()
    df_clone = df_clone.dropna(subset=['athlete_full_name'])
    df_clone = df_clone[df_clone['athlete_full_name'] == name]

    # Select the desired columns
    filtered_df = df_clone[['slug_game', 'event_title', 'athlete_full_name', 'medal_type']]

    # Group the DataFrame by 'slug_game' and 'medal_type' and count the occurrences
    grouped_df = filtered_df.groupby(['slug_game', 'medal_type']).size().unstack().fillna(0)

    # Create a list of medal types
    medal_types = grouped_df.columns

    # Create a trace for each medal type
    traces = []
    for medal_type in medal_types:
        trace = go.Bar(
            x=grouped_df.index,
            y=grouped_df[medal_type],
            name=medal_type
        )
        traces.append(trace)

    # Create the layout
    layout = go.Layout(
        title=f"Medal Count by Venue Name for {name}",
        xaxis=dict(title='Venue Name'),
        yaxis=dict(title='Medal Count'),
        barmode='stack'
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)

    # Display the interactive plot
    fig.show()

athlete1 = input("Enter the name of the first athlete: ").strip()
athlete2 = input("Enter the name of the second athlete: ").strip()

create_medal_count_plot(athlete1, df)
create_medal_count_plot(athlete2, df)

###PREVELENCE OF DISCIPLINES ACROSS OLYMPIC HISTORY

In [19]:
import pandas as pd
import plotly.graph_objects as go

def generate_bar_plot(dataframe, num_disciplines=None):
    # Group the DataFrame by 'discipline_title' and count the occurrences
    grouped_df = dataframe.groupby('discipline_title').size().reset_index(name='count')

    # Sort the disciplines by count in descending order
    sorted_df = grouped_df.sort_values(by='count', ascending=False)

    # Select the top disciplines based on the specified number
    if num_disciplines:
        top_df = sorted_df.head(num_disciplines)
    else:
        top_df = sorted_df

    # Create the bar trace
    bar_trace = go.Bar(
        x=top_df['discipline_title'],
        y=top_df['count'],
        name='Count'
    )

    # Create the layout
    layout = go.Layout(
        title=f'Bar Plot of Top {num_disciplines} Disciplines' if num_disciplines else 'Bar Plot of All Disciplines',
        xaxis=dict(title='Discipline Title'),
        yaxis=dict(title='Count')
    )

    # Create the figure
    fig = go.Figure(data=[bar_trace], layout=layout)

    # Display the interactive plot
    fig.show()

# Assuming you have a DataFrame named 'df' available
generate_bar_plot(df, num_disciplines=5)  # Top 5 disciplines
generate_bar_plot(df, num_disciplines=10)  # Top 10 disciplines
generate_bar_plot(df)  # All disciplines


In [20]:
import pandas as pd
import plotly.graph_objects as go

def generate_bar_plot(dataframe, num_disciplines, disciplines):
    # Filter the DataFrame based on the selected disciplines
    filtered_df = dataframe[dataframe['discipline_title'].isin(disciplines)]

    # Group the filtered DataFrame by 'discipline_title' and count the occurrences
    grouped_df = filtered_df.groupby('discipline_title').size().reset_index(name='count')

    # Sort the disciplines by count in descending order
    sorted_df = grouped_df.sort_values(by='count', ascending=False)

    # Select the top disciplines based on the specified number
    top_df = sorted_df.head(num_disciplines)

    # Create the bar trace
    bar_trace = go.Bar(
        x=top_df['discipline_title'],
        y=top_df['count'],
        name='Count'
    )

    # Create the layout
    layout = go.Layout(
        title=f'Bar Plot of Top {num_disciplines} Disciplines',
        xaxis=dict(title='Discipline Title'),
        yaxis=dict(title='Count')
    )

    # Create the figure
    fig = go.Figure(data=[bar_trace], layout=layout)

    # Display the interactive plot
    fig.show()

# Assuming you have a DataFrame named 'df' available
num_disciplines = int(input("Enter the number of disciplines to compare: "))
disciplines = []
for i in range(num_disciplines):
    discipline = input(f"Enter discipline {i+1} name: ")
    disciplines.append(discipline)

generate_bar_plot(df, num_disciplines, disciplines)


### HISTORIC MEDAL WINS FOR EACH COUNTRY FOR A CERTAIN EVENT IN A CERTAIN DISCIPLINE

In [21]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt

def get_medal_counts(discipline_title, event_title):
    # Filter the DataFrame based on discipline_title
    dis_df = df[df["discipline_title"] == discipline_title]

    # Filter the DataFrame based on event_title
    dis_df = dis_df[dis_df["event_title"] == event_title]

    # Group the filtered DataFrame by 'country_name' and 'medal_type', count the occurrences
    grouped_df = dis_df.groupby(['country_name', 'medal_type']).size().reset_index(name='count')

    # Pivot the grouped DataFrame to convert medal types into separate columns
    pivot_df = grouped_df.pivot(index='country_name', columns='medal_type', values='count')

    # Fill NaN values with zero
    pivot_df.fillna(0, inplace=True)

    # Reset the index of the DataFrame
    pivot_df.reset_index(inplace=True)

    # Calculate the total medal count
    pivot_df['Total'] = pivot_df['GOLD'] + pivot_df['SILVER'] + pivot_df['BRONZE']

    # Sort the DataFrame in descending order based on the total medal count
    sorted_df = pivot_df.sort_values(by='Total', ascending=False)

    # Reset the index of the sorted DataFrame
    sorted_df.reset_index(drop=True, inplace=True)

    # Return the resulting DataFrame
    return sorted_df
# Call the function with the desired parameters
discipline_title = input("Enter discipline title: ")
event_title = input("Enter event title: ")
result_df = get_medal_counts(discipline_title,event_title)

import plotly.graph_objects as go

import plotly.graph_objects as go

def generate_grouped_bar_plot(result_df):
    # Create a trace for each medal type
    trace_gold = go.Bar(x=result_df['country_name'], y=result_df['GOLD'], name='Gold', marker=dict(color='gold'))
    trace_silver = go.Bar(x=result_df['country_name'], y=result_df['SILVER'], name='Silver', marker=dict(color='silver'))
    trace_bronze = go.Bar(x=result_df['country_name'], y=result_df['BRONZE'], name='Bronze', marker=dict(color='peru'))

    # Create the data list
    data = [trace_gold, trace_silver, trace_bronze]

    # Set the layout
    layout = go.Layout(
        title='Grouped Bar Plot of Medal Counts',
        xaxis=dict(title='Country'),
        yaxis=dict(title='Medal Count'),
        barmode='group'
    )

    # Create the figure
    fig = go.Figure(data=data, layout=layout)

    # Show the figure
    fig.show()

generate_grouped_bar_plot(result_df)

In [22]:
import pandas as pd
import plotly.graph_objects as go

def get_medal_counts(df,discipline_title, slug_name):
    # Filter the DataFrame based on discipline_title
    dis_df = df[df["discipline_title"] == discipline_title]


    # Group the filtered DataFrame by 'country_name' and 'medal_type', count the occurrences
    grouped_df = dis_df.groupby(['country_name', 'medal_type']).size().reset_index(name='count')

    # Pivot the grouped DataFrame to convert medal types into separate columns
    pivot_df = grouped_df.pivot(index='country_name', columns='medal_type', values='count')

    # Fill NaN values with zero
    pivot_df.fillna(0, inplace=True)

    # Reset the index of the DataFrame
    pivot_df.reset_index(inplace=True)

    # Calculate the total medal count
    pivot_df['Total'] = pivot_df['GOLD'] + pivot_df['SILVER'] + pivot_df['BRONZE']

    # Sort the DataFrame in descending order based on the total medal count
    sorted_df = pivot_df.sort_values(by='Total', ascending=False)

    # Reset the index of the sorted DataFrame
    sorted_df.reset_index(drop=True, inplace=True)

    # Return the resulting DataFrame
    return sorted_df

# Call the function with the desired parameters
discipline_title = input("Enter discipline title: ")
slug_name = input("Enter slug name: ")
result_df = get_medal_counts(df,discipline_title, slug_name)

def generate_grouped_bar_plot(result_df,discipline_title,slug_name):
    # Create a trace for each medal type
    trace_gold = go.Bar(x=result_df['country_name'], y=result_df['GOLD'], name='Gold', marker=dict(color='gold'))
    trace_silver = go.Bar(x=result_df['country_name'], y=result_df['SILVER'], name='Silver', marker=dict(color='silver'))
    trace_bronze = go.Bar(x=result_df['country_name'], y=result_df['BRONZE'], name='Bronze', marker=dict(color='peru'))

    # Create the data list
    data = [trace_gold, trace_silver, trace_bronze]

    # Set the layout
    layout = go.Layout(
        title=f"Grouped Bar Plot Of Medal Counts For Discipline {discipline_title} In Venue {slug_name} ",
        xaxis=dict(title='Country'),
        yaxis=dict(title='Medal Count'),
        barmode='group'
    )

    # Create the figure
    fig = go.Figure(data=data, layout=layout)

    # Show the figure
    fig.show()

generate_grouped_bar_plot(result_df,discipline_title,slug_name)
